In [1]:
import pandas as pd
import os
from sys import platform
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pickle
import seaborn

### Load Data

In [2]:
df = pd.read_csv("/Users/emilywen/Documents/CMU MSBA/Machine Learning for Business Applications/Project/Telecommunications_Industry/Telco_customer_churn.csv")
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


### Pre-processing

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerID         7043 non-null   object 
 1   Count              7043 non-null   int64  
 2   Country            7043 non-null   object 
 3   State              7043 non-null   object 
 4   City               7043 non-null   object 
 5   Zip Code           7043 non-null   int64  
 6   Lat Long           7043 non-null   object 
 7   Latitude           7043 non-null   float64
 8   Longitude          7043 non-null   float64
 9   Gender             7043 non-null   object 
 10  Senior Citizen     7043 non-null   object 
 11  Partner            7043 non-null   object 
 12  Dependents         7043 non-null   object 
 13  Tenure Months      7043 non-null   int64  
 14  Phone Service      7043 non-null   object 
 15  Multiple Lines     7043 non-null   object 
 16  Internet Service   7043 

#### Set column types

In [4]:
df['Total Charges'] = pd.to_numeric(df['Total Charges'], errors='coerce')

#### Check Null Value

In [5]:
df.isnull().sum()

CustomerID              0
Count                   0
Country                 0
State                   0
City                    0
Zip Code                0
Lat Long                0
Latitude                0
Longitude               0
Gender                  0
Senior Citizen          0
Partner                 0
Dependents              0
Tenure Months           0
Phone Service           0
Multiple Lines          0
Internet Service        0
Online Security         0
Online Backup           0
Device Protection       0
Tech Support            0
Streaming TV            0
Streaming Movies        0
Contract                0
Paperless Billing       0
Payment Method          0
Monthly Charges         0
Total Charges          11
Churn Label             0
Churn Value             0
Churn Score             0
CLTV                    0
Churn Reason         5174
dtype: int64

#### Drop columns

In [6]:
df = df.drop(['CustomerID', 'Count', 'Country', 'State', 'City', 'Lat Long', 'Latitude', 'Longitude', 'Churn Label', 'Churn Score', 'Churn Reason'], axis=1)

#### One-hot encoding

In [7]:
df = pd.get_dummies(df, drop_first=True)
df

,Zip Code,Tenure Months,Monthly Charges,Total Charges,Churn Value,CLTV,Gender_Male,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,...,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes,Contract_One year,Contract_Two year,Paperless Billing_Yes,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
0,90003,2,53.85,108.15,1,3239,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,90005,2,70.70,151.65,1,2701,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
2,90006,8,99.65,820.50,1,5372,0,0,0,1,...,0,1,0,1,0,0,1,0,1,0
3,90010,28,104.80,3046.05,1,5003,0,0,1,1,...,0,1,0,1,0,0,1,0,1,0
4,90015,49,103.70,5036.30,1,5340,1,0,0,1,...,0,1,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,92285,72,21.15,1419.40,0,5306,0,0,0,0,...,1,0,1,0,0,1,1,0,0,0
7039,92301,24,84.80,1990.50,0,2140,1,0,1,1,...,0,1,0,1,1,0,1,0,0,1
7040,92304,72,103.20,7362.90,0,5560,0,0,1,1,...,0,1,0,1,1,0,1,1,0,0
7041,92305,11,29.60,346.45,0,2793,0,0,1,1,...,0,0,0,0,0,0,1,0,1,0


In [12]:
df.isnull().sum()

Zip Code                                   0
Tenure Months                              0
Monthly Charges                            0
Total Charges                             11
Churn Value                                0
CLTV                                       0
Gender_Male                                0
Senior Citizen_Yes                         0
Partner_Yes                                0
Dependents_Yes                             0
Phone Service_Yes                          0
Multiple Lines_No phone service            0
Multiple Lines_Yes                         0
Internet Service_Fiber optic               0
Internet Service_No                        0
Online Security_No internet service        0
Online Security_Yes                        0
Online Backup_No internet service          0
Online Backup_Yes                          0
Device Protection_No internet service      0
Device Protection_Yes                      0
Tech Support_No internet service           0
Tech Suppo

In [13]:
df = df.dropna()

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [14]:
y = df['Churn Value']
X = df.drop('Churn Value',axis=1)

# Stratified splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=2024)

In [15]:
# Train a logistic regression model
clf = LogisticRegression(penalty=None, max_iter=1000)
clf.fit(X_train,y_train)

y_pred = clf.predict_proba(X_test)[:,1]

# Compute the AUC
roc_auc_score(y_test,y_pred)

/Users/emilywen/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8628291037055704